In [1]:
import pandas as pd

import os
import re

from pathlib import Path
from typing import *

import sys
sys.path.append('..')

from src import utils

%load_ext autoreload
%autoreload 2 

In [2]:
import src
from src.tables import tables
from src.core import run, Params

In [3]:
sheet_name = 'rep_hiv_avrt_analyze'
input_path = '../examples/input.xlsx'
output_path = '../outputs/out.xlsx'

In [34]:
run(
    Params(
        sheet_name='rep_hiv_avrt_analyze',
        input_path='../examples/input.xlsx',
        output_path='../outputs/out2.xlsx',
        pivot_tables=tables,
    )
)

C:\Miniconda\envs\tables\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [72]:
table = tables[-1]
table

PivotTable(name='Потребность в препаратах', fields=Fields(values=(Value(field='УНРЗ', name='Количество по полю УНРЗ', calculation=<Calculation.COUNT: 'count'>, number_format='0'), Value(field='Потребность на год (ЕИ)', name='Сумма по полю Потребность на год (ЕИ)', calculation=<Calculation.SUM: 'sum'>, number_format='0')), rows=('МНН+Дозировка',), columns=(), filters=()))

In [73]:
table.name

'Потребность в препаратах'

In [74]:
import win32com.client as win32

In [103]:
# create Excel object
excel = win32.gencache.EnsureDispatch('Excel.Application')
# excel can be visible or not
excel.Visible = True 
# open Excel Workbook   
wb = excel.Workbooks.Open(Path(output_path).resolve())

In [76]:
win32c = win32.constants

In [77]:
class PTField(NamedTuple):
    field: str
    name: str
    calculation: object
    number_format: str = '0'

In [78]:
class PTFields(NamedTuple):
    values: List[PTField]
    rows: List[str] = tuple()
    columns: List[str] = tuple()
    filters: List[str] = tuple()

In [79]:
def create_new_sheet(wb: object, name: str) -> object:
    wb.Sheets.Add().Name = name
    return wb.Sheets(name)

In [80]:
def pivot_table(wb: object, ws: object, pt_ws: object, fields: PTFields, show_annotations: bool = False):
    """
    wb = workbook1 reference
    ws = worksheet1
    pt_ws = pivot table worksheet
    ws_name = pivot table worksheet name
    pt_name = name given to pivot table
    pt_rows, pt_cols, pt_filters, pt_fields: values selected for filling the pivot tables
    """

    # pivot table location
    pt_loc = len(fields.filters) + 2
    
    # grab the pivot table source data
    pc = wb.PivotCaches().Create(SourceType=win32c.xlDatabase, SourceData=ws.UsedRange)
    
    # create the pivot table object
    pc.CreatePivotTable(TableDestination=f"'{pt_ws.Name}'!R{pt_loc}C1", TableName=pt_ws.Name)

    # selecte the pivot table work sheet and location to create the pivot table
    pt_ws.Select()
    pt_ws.Cells(pt_loc, 1).Select()

    # Sets the rows, columns and filters of the pivot table
    for field_list, field_r in ((fields.filters, win32c.xlPageField), (fields.rows, win32c.xlRowField), (fields.columns, win32c.xlColumnField)):
        for i, value in enumerate(field_list):
            pt_ws.PivotTables(pt_ws.Name).PivotFields(value).Orientation = field_r
            pt_ws.PivotTables(pt_ws.Name).PivotFields(value).Position = i + 1

    # Sets the Values of the pivot table
    for value in fields.values:
        field = (
            pt_ws.PivotTables(pt_ws.Name)
            .AddDataField(
                pt_ws.PivotTables(pt_ws.Name).PivotFields(value.field), 
                value.name, 
                value.calculation
            )
        )
        field.NumberFormat = value.number_format

    # Visiblity True or Valse
    pt_ws.PivotTables(pt_ws.Name).ShowValuesRow = show_annotations
    pt_ws.PivotTables(pt_ws.Name).ColumnGrand = show_annotations

In [102]:
wb.Close(True)

In [104]:
ws = excel.Sheets('Исходный лист')

In [108]:
src.excel.create_pivot_table(wb, ws, table)

# Done

In [67]:
out_1 = inp.copy()
out_1['МНН+Дозировка'] = inp[['МНН', 'Дозировка']].apply(lambda r: ', '.join((r.МНН, r.Дозировка)), axis=1)

In [11]:
inverse_columns_map = {v: k for k, v in columns_map.items()}
agg_columns_map = {
    'Количество по полю УНРЗ': ('УНРЗ', 'count'),
    'Сумма по полю Потребность на год (ЕИ)': ('ПотребностьНаГодЕИ', 'sum'),
}

out_1 = (out_1
    .groupby('МНН+Дозировка')
    .agg(**agg_columns_map)
    .reset_index()
)

In [48]:
out_2 = inp.copy()
out_2['МНН+Дозировка'] = inp[['МНН', 'Дозировка']].apply(lambda r: ', '.join((r.МНН, r.Дозировка)), axis=1)

In [49]:
out_2 = (
    out_2
    .groupby('УНРЗ')
    .agg({'МНН+Дозировка': lambda v: '; '.join(v)})
    .reset_index()
    .groupby('МНН+Дозировка')
    .agg({'УНРЗ': ('count', ('Список', lambda v: ', '.join(v)))})
    .reset_index()
    .rename(columns={'count': 'Количество'})
    .reset_index(drop=True)
)
out_2

МНН+Дозировка       УНРЗ  \
                                                       Количество   
0    Абакавир + Ламивудин, 600 мг + 300 мг; Атазана...          7   
1    Абакавир + Ламивудин, 600 мг + 300 мг; Атазана...          4   
2    Абакавир + Ламивудин, 600 мг + 300 мг; Дарунав...          5   
3    Абакавир + Ламивудин, 600 мг + 300 мг; Долутег...          3   
4    Абакавир + Ламивудин, 600 мг + 300 мг; Лопинав...         21   
..                                                 ...        ...   
116  Ламивудин, 300 мг; Фосфазид, 400 мг; Эфавиренз...          1   
117  Лопинавир + Ритонавир, 200 мг + 50 мг; Ралтегр...          2   
118  Лопинавир + Ритонавир, 200 мг + 50 мг; Тенофов...          1   
119  Рилпивирин+ Тенофовир+ Эмтрицитабин, 25 мг + 3...        193   
120  Тенофовир, 300 мг; Эмтрицитабин, 200 мг; Эфави...         13   

                                                        
                                                Список  
0    7010118818801, 7010217005713, 7010313736435, 7...  
1    7010217009003, 7010217012773, 7010217012779, 7...  
2    6610617385511, 7010617253064, 7020617618031, 7...  
3          7010517101928, 7010618878384, 7010816607007  
4    7010118818799, 7010211637908, 7010217005038, 7...  
..                                                 ...  
116                                      7010612576990  
117                     7010117625856, 701022001049565  
118                                      7010515426736  
119  2220513757831, 7010113728932, 7010117626077, 7...  
120  7010118817846, 7010119936308, 701022001051866,...  

[121 rows x 3 columns]

In [42]:
out_2.set_index('МНН+Дозировка')

УНРЗ  \
                                                   Количество   
МНН+Дозировка                                                   
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...          7   
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...          4   
Абакавир + Ламивудин, 600 мг + 300 мг; Дарунави...          5   
Абакавир + Ламивудин, 600 мг + 300 мг; Долутегр...          3   
Абакавир + Ламивудин, 600 мг + 300 мг; Лопинави...         21   
...                                                       ...   
Ламивудин, 300 мг; Фосфазид, 400 мг; Эфавиренз,...          1   
Лопинавир + Ритонавир, 200 мг + 50 мг; Ралтегра...          2   
Лопинавир + Ритонавир, 200 мг + 50 мг; Тенофови...          1   
Рилпивирин+ Тенофовир+ Эмтрицитабин, 25 мг + 30...        193   
Тенофовир, 300 мг; Эмтрицитабин, 200 мг; Эфавир...         13   

                                                                                                       
                                                                                          Список УНРЗ  
МНН+Дозировка                                                                                          
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...  7010118818801, 7010217005713, 7010313736435, 7...  
Абакавир + Ламивудин, 600 мг + 300 мг; Атазанав...  7010217009003, 7010217012773, 7010217012779, 7...  
Абакавир + Ламивудин, 600 мг + 300 мг; Дарунави...  6610617385511, 7010617253064, 7020617618031, 7...  
Абакавир + Ламивудин, 600 мг + 300 мг; Долутегр...        7010517101928, 7010618878384, 7010816607007  
Абакавир + Ламивудин, 600 мг + 300 мг; Лопинави...  7010118818799, 7010211637908, 7010217005038, 7...  
...                                                                                               ...  
Ламивудин, 300 мг; Фосфазид, 400 мг; Эфавиренз,...                                      7010612576990  
Лопинавир + Ритонавир, 200 мг + 50 мг; Ралтегра...                     7010117625856, 701022001049565  
Лопинавир + Ритонавир, 200 мг + 50 мг; Тенофови...                                      7010515426736  
Рилпивирин+ Тенофовир+ Эмтрицитабин, 25 мг + 30...  2220513757831, 7010113728932, 7010117626077, 7...  
Тенофовир, 300 мг; Эмтрицитабин, 200 мг; Эфавир...  7010118817846, 7010119936308, 701022001051866,...  

[121 rows x 2 columns]

In [33]:
# todo:
# 1. сделать вывод результатов файл с тремя страницами
# 2. сделать простенький GUI:
#    - выбор исходного файла и названием целевого листа на нем
#    - окошко со статусом преобразования / какой-то информацией об ошибках
#    - выбор пути для сохранения нового файла

out_path = Path('../outputs/')
out_path.mkdir(parents=True, exist_ok=True)
out_file = out_path / 'out.xlsx'

try:
    os.remove(out_file)
except FileNotFoundError:
    pass

In [28]:
out_1.to_excel(
    out_file,
    sheet_name='Потребность в препаратах',
    # engine=engine,
)
out_2.to_excel(
    out_file,
    sheet_name='Схемы',
    # engine=engine,
)
inp.to_excel(
    out_file,
    sheet_name=sheet_name,
    # engine=engine,
)

In [ ]:
inp

In [36]:
from pandas import ExcelWriter
from typing import Dict


def save_xls(sheets: Dict[str, pd.DataFrame], path: str, **kwargs):
    """Save a dictionary of dataframes to an excel file, with each dataframe as a separate page."""

    with ExcelWriter(path) as writer:
        for sheet, data in sheets.items():
            data.to_excel(excel_writer=writer, sheet_name=sheet, **kwargs)

In [37]:
save_xls(
    sheets={
        'Потребность в препаратах': out_1,
        'Схемы': out_2,
        sheet_name: inp.rename(columns={
            v: k for k, v in columns_map.items()
        })
    },
    path=out_file,
    index=False
)

NotImplementedError: Writing to Excel with MultiIndex columns and no index ('index'=False) is not yet implemented.

## GUI

In [11]:
import tkinter as tk
from tkinter import ttk

In [5]:
import tkinter
from tkinter import messagebox

app = tkinter.Tk()
app.title("22_tkinter_advanced.py")
app.geometry("600x600")

check_widget = tkinter.Checkbutton(app, text="scan_port", offvalue=2, onvalue=5)
check_widget.pack()

radio_widgetH = tkinter.Radiobutton(app, text="Homme", value=2)
radio_widgetF = tkinter.Radiobutton(app, text="Femme", value=1)
radio_widgetH.pack()
radio_widgetF.pack()

scale_widget = tkinter.Scale(app, from_=10, to=50, tickinterval=4+1)
scale_widget.pack()

spin_widget = tkinter.Spinbox(app, from_=5, to=100)
spin_widget.pack()

listbox_widget = tkinter.Listbox(app)
listbox_widget.insert(1, "Windows")
listbox_widget.insert(2, "MacOS")
listbox_widget.insert(3, "GNU/Linux")
listbox_widget.pack()

def show_modal_windows_error():
    messagebox.showerror(title="ERREUR", message="Un probelème est survenu !")

button_error = tkinter.Button(app, text="Déclancheur D'érreur", command=show_modal_windows_error)
button_error.pack()

def show_modal_windows_info():
    messagebox.showinfo(title="INFO", message="Une inforamtion fort utiles ;)")

button_info = tkinter.Button(app, text="Déclancheur d'info", command=show_modal_windows_info)
button_info.pack()

def show_modal_windows_warning():
    messagebox.showwarning(title="WARNING", message="Ceci est un avertissement")

button_warning = tkinter.Button(app, text="Déclancheur d'alerte", command=show_modal_windows_warning)
button_warning.pack()

def show_modal_windows_askquestion():
    messagebox.askquestion(title="Confirm", message="Etes vous vraiments sûr ?")

button_askquestion = tkinter.Button(app, text="askquestion", command=show_modal_windows_askquestion)
button_askquestion.pack()

def show_modal_windows_askokcancel():
    messagebox.askokcancel(title="Confirm", message="Voulez vous vraiment quittez ?")

button_askokcancel = tkinter.Button(app, text="askokcancel", command=show_modal_windows_askokcancel)
button_askokcancel.pack()

def show_modal_windows_askyesno():
    messagebox.askyesno(title="Confirm", message="C'est OUI où bien c'est NON")

button_askyesno = tkinter.Button(app, text="askyesno", command=show_modal_windows_askyesno)
button_askyesno.pack()

def show_modal_windows_askretrycancel():
    messagebox.askretrycancel(title="Retry ?", message="Voulez vous réesayez ?")

button_askretrycancel = tkinter.Button(app, text="askretrycancel", command=show_modal_windows_askretrycancel)
button_askretrycancel.pack()

app.mainloop()

In [4]:
class Placeholder:
    
    def __init__(self, master, placeholder='', placeholdercolor='grey', color='black', **kwargs):
        self.e = ttk.Entry(master, foreground=placeholdercolor, **kwargs)
        self.e.bind('<FocusIn>', self.focus_in)
        self.e.bind('<FocusOut>', self.focus_out)
        self.e.insert(0, placeholder)
        self.placeholder = placeholder
        self.placeholdercolor = placeholdercolor
        self.color = color

    def pack(self, side=None, **kwargs):
        self.e.pack(side=side, **kwargs)

    def place(self, side=None, **kwargs):
        self.e.place(side=side, **kwargs)

    def grid(self, column=None, **kwargs):
        self.e.grid(column=column, **kwargs)

    def focus_in(self, e):
        if self.e.get() == self.placeholder:
            self.e.delete(0, tk.END)
        self.e.configure(foreground=self.color)

    def focus_out(self, e):
        if self.e.get() == '':
            self.e.configure(foreground=self.placeholdercolor)
            self.e.delete(0, tk.END)
            self.e.insert(0, self.placeholder)

In [50]:
from ctypes import windll
windll.shcore.SetProcessDpiAwareness(1)

window = tk.Tk()
window.title('Создание сводных отчетов')
# window.geometry('344x244')
window.resizable(False, False)

frame = ttk.Frame(window, padding=12)
frame.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))

window.columnconfigure(0, weight=1)
window.rowconfigure(0, weight=1)

window.columnconfigure(1, weight=1)
window.rowconfigure(1, weight=1)

window.columnconfigure(2, weight=1)
window.rowconfigure(2, weight=1)

input_label = ttk.Label(frame, text='Путь до исходного файла', justify=tk.LEFT)
input_label.grid(row=0, column=0, sticky=tk.W)

input_path = tk.StringVar()
input_entry = Placeholder(frame, justify=tk.LEFT, textvariable=input_path, width=32)
input_entry.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E))

sheet_name = tk.StringVar()
sheet_label = ttk.Label(frame, text='Лист')
sheet_label.grid(row=0, column=2, sticky=tk.W)

sheet_entry = Placeholder(frame, placeholder='необязательно', justify=tk.LEFT, textvariable=sheet_name, width=16)
sheet_entry.grid(row=1, column=2, sticky=tk.W)

output_label = ttk.Label(frame, text='Путь сохранения результата', justify=tk.LEFT)
output_label.grid(row=2, column=0, columnspan=2, sticky=tk.W)

output_path = tk.StringVar()
output_entry = Placeholder(frame, justify=tk.LEFT, textvariable=output_path, width=32)
output_entry.grid(row=3, column=0, columnspan=2, sticky=(tk.W, tk.E))

tables_label_frame = ttk.LabelFrame(frame, text='Отчеты')
tables_label_frame.grid(row=4, column=0, columnspan=2, sticky=(tk.W, tk.E), ipadx=2, ipady=10)

for child in frame.winfo_children(): 
    child.grid_configure(padx=4, pady=2)

for idx, table in enumerate(('Потребность в препаратах', 'Схемы лечения')):
    ttk.Checkbutton(tables_label_frame, text=table, onvalue=True).grid(row=5 + idx, column=0, sticky=tk.W)

start = ttk.Button(frame, text='Запуск')
start.grid(row=7, column=1, sticky=(tk.W, tk.E), padx=4, pady=16)

cancel = ttk.Button(frame, text='Закрыть')
cancel.grid(row=7, column=2, sticky=(tk.W, tk.E), padx=4, pady=16)

start.configure(width=16)

window.mainloop()

In [5]:
from src.tables import tables
from src.gui import GUI

In [ ]:
tk.f

In [7]:
class TkinterGUI(GUI):
    
    def configure(self) -> None:
        self.input_entry.configure()
    
    def create(self) -> GUI:
        from ctypes import windll
        windll.shcore.SetProcessDpiAwareness(1)

        self.window = tk.Tk()
        self.window.title('Создание сводных отчетов')
        self.window.resizable(False, False)

        self.frame = ttk.Frame(self.window, padding=12)
        self.frame.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))

        self.window.columnconfigure(0, weight=1)
        self.window.rowconfigure(0, weight=1)

        self.window.columnconfigure(1, weight=1)
        self.window.rowconfigure(1, weight=1)

        self.window.columnconfigure(2, weight=1)
        self.window.rowconfigure(2, weight=1)

        self.input_label = ttk.Label(self.frame, text='Путь до исходного файла', justify=tk.LEFT)
        self.input_label.grid(row=0, column=0, sticky=tk.W)

        self.input_path = tk.StringVar()
        self.input_entry = Placeholder(self.frame, justify=tk.LEFT, textvariable=self.input_path, width=32)
        self.input_entry.grid(row=1, column=0, columnspan=2, sticky=(tk.W, tk.E))

        self.sheet_name = tk.StringVar()
        self.sheet_label = ttk.Label(self.frame, text='Лист')
        self.sheet_label.grid(row=0, column=2, sticky=tk.W)

        self.sheet_entry = Placeholder(self.frame, placeholder='необязательно', justify=tk.LEFT, textvariable=self.sheet_name, width=16)
        self.sheet_entry.grid(row=1, column=2, sticky=tk.W)

        self.output_label = ttk.Label(self.frame, text='Путь сохранения результата', justify=tk.LEFT)
        self.output_label.grid(row=2, column=0, columnspan=2, sticky=tk.W)

        self.output_path = tk.StringVar()
        self.output_entry = Placeholder(self.frame, justify=tk.LEFT, textvariable=self.output_path, width=32)
        self.output_entry.grid(row=3, column=0, columnspan=2, sticky=(tk.W, tk.E))

        self.tables_label_frame = ttk.LabelFrame(self.frame, text='Отчеты')
        self.tables_label_frame.grid(row=4, column=0, columnspan=2, sticky=(tk.W, tk.E), ipadx=2, ipady=10)

        for child in self.frame.winfo_children(): 
            child.grid_configure(padx=4, pady=2)

        for idx, table in enumerate(self._available_tables):
            ttk.Checkbutton(self.tables_label_frame, text=table.name, onvalue=True).grid(row=5 + idx, column=0, sticky=tk.W)

        self.start = ttk.Button(self.frame, text='Запуск', width=16)
        self.start.grid(row=7, column=1, sticky=(tk.W, tk.E), padx=4, pady=16)

        self.cancel = ttk.Button(self.frame, text='Закрыть')
        self.cancel.grid(row=7, column=2, sticky=(tk.W, tk.E), padx=4, pady=16)

        self.window.mainloop()

In [10]:
with TkinterGUI(tables) as gui:
    pass